In [1]:
# Import Dependencies

from time import sleep
# from features import get_features
from mido import MidiFile
import pandas as pd

from datetime import date
from time import time, monotonic
from os import path, mkdir, listdir, environ
import subprocess


In [2]:
# Get Args, define evaluation groups

iterations = 4        # How many times each experiment should be repeated for the same parameters?
max_input_bars = 2    # What is the maximum lookback size for each model?
max_output_bars = 2   # What is the maximum output size for each model?
# sample_length = 32    # How long of a final composition should be generated?
sample_length = 8     # How long of a final composition should be generated?

# TODO: Remove this quick test block
iterations = 1
max_input_bars = 2
max_output_bars = 2
# sample_length = 32    # How long of a final composition should be generated?
sample_length = 1
# / TODO: Remove this quick test block

client  = None        # Client Process
server  = None        # Server Process
expname = ''          # Experiment Name, for naming files

ip = "127.0.0.1"
port = 5005
port_in = 57120

modelname = None
checkpointname = None

evaluation_sets = [
  ('bach-chorales', [
    ('polyphony_rnn', 'polyphony_rnn'),
    ('pianoroll_rnn_nade', 'rnn-nade_attn'),
    ('rl_duet', 'rl_duet'),
  ]),
  ('piano-e-comp', [
    ('performance_rnn', 'performance_with_dynamics'),
    # ('music_transformer', 'performance_with_dynamics')
  ]),
  (None, [
    # ('melody_rnn', 'attention_rnn')
  ]),
  (None, [
    # ('remi', 'remi')
  ])
]
cols_gen = [ "model", "checkpoint", "dataset", "primer", "out_file", "time", "in_len", "out_len" ]

In [3]:
# Folder Definitions
primerdir = ''
# outputdir = path.join(path.pardir,'output')
# datasetdir = path.join(path.pardir,'dataset') 
outputdir = path.join(path.curdir,'output')
datasetdir = path.join(path.curdir,'dataset')
basefoldername = str(date.today())
i = 0
while True:
    foldername = f'{basefoldername}-{i}'
    full_foldername = path.join(outputdir, foldername)

    if not path.exists(full_foldername):
      mkdir(full_foldername)
      break

    if not any(listdir(full_foldername)): break
    i = i + 1


# File Management
def get_filename(expname,index,primer=None):
  return path.normpath(
    f"{foldername}/{expname}-{index}"
    if primer is None else
    f"{foldername}/{expname}-{primer}-{index}")

def get_primer_filename(name):
  return path.join(primerdir,name)

def get_midi_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.mid')

def get_pickle_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.pkl')

def save_df(df, filename):
    # log(f'Saving dataframe to {filename}')
    df.to_pickle(filename)

def log(message):
    print(f'[batch:{expname}] {message}')

In [4]:
# Define Experiments

def generate_sample(primer, bars_input, bars_output, index):
  client.reset()
  client.load_bars(get_primer_filename(primer), bars_input)

  i = 0
  start_time = monotonic()
  while bars_input + i * bars_output < sample_length:
    client.generate(bars_output, 'measures')
    i += 1
  gentime = monotonic() - start_time

  filename = get_filename('generation',index,f'{start_time}-{bars_input}-{bars_output}-{primer}')
  client.save(filename)
  return filename, gentime

def run_generation(primer, i):
  return [[None, None, None, primer, *generate_sample(primer, bars_input, bars_output, i), bars_input, bars_output]
    for bars_input
    in range(1,max_input_bars + 1)
    for bars_output
    in range(1,max_output_bars + 1)
  ]

experiments = [
  # ('input_len', analysis_input_length),
  # ('output_len', analysis_output_length),
  ('generation', run_generation)
  ]

def experiment():
  pp = [run_generation(primer, i) for i in range(iterations) for primer in primers]
  out = []
  for p in pp:
    out += p
  return p

In [5]:
# Start Client, define model management functions
from pythonosc import udp_client, osc_server
from pythonosc.dispatcher import Dispatcher

class BatchClient(udp_client.SimpleUDPClient):
  def __init__(self, logger, ip, port_out, port_in):
    udp_client.SimpleUDPClient.__init__(self, ip, port_out, port_in)
    dispatcher = Dispatcher()
    dispatcher.map('/ok', lambda _: self.server.shutdown())
    self.server = osc_server.ThreadingOSCUDPServer((ip, port_in), dispatcher)
    self.log = logger

  def request(self, endpoint, args):
    self.log(f'{endpoint} {" ".join([str(a) for a in args])}')
    self.send_message(endpoint, args)

  def start(self):
    self.request('/start', [])

  def stop(self):
    self.request('/stop', [])

  def set(self,*args):
    self.request('/set', args)

  def debug(self,key):
    self.request('/debug', [key])

  def save(self,filename):
    self.request('/save', [filename])
    self.wait()

  def play(self,note):
    self.request('/play', [note + 40])

  def run(self,filename):
    self.set('output_filename', filename)
    self.start() # TODO: on host: unset 'batch_complete'
    self.wait()

  def reset(self):
    self.request('/reset', [])
  
  def pause(self):
    self.request('/pause', [])

  def end(self):
    self.request('/end', [])

  def generate(self, length, unit):
    self.request('/generate', [length, unit])
    self.wait()

  def wait(self):
    self.log("waiting...")
    self.server.serve_forever()
    self.log("ok!")
  
  def load_bars(self,filename,barcount):
    self.request('/load_bars', [filename, barcount])
    self.wait()

client = BatchClient(log, ip,  port,  port_in)
logfile = None
def start_model(the_modelname, the_checkpointname):
    logfile = open(path.join(outputdir, f'{the_modelname}.log'), 'w')
    modelname = the_modelname
    checkpointname = the_checkpointname
    log(f'Starting model: {modelname} ({checkpointname})')
    env = environ.copy()
    env['NOT_INTERACTIVE'] = '1'
    # server = subprocess.Popen([ './start.sh', model, checkpoint, 'batch' ], cwd=path.pardir, env=env)
    server = subprocess.Popen([ './start.sh', modelname, checkpointname, 'batch' ], env=env, stdout=logfile, stderr=logfile)
    # server.communicate()
    client.wait()
    client.set('debug_output', False)
    client.set('batch_mode', True)
    client.set('trigger_generate', 1)
    client.set('batch_unit', 'measures')
    client.set('debug_output', False)

def stop_model():
  if client: client.end()
  if server: server.wait()
  if logfile: logfile.close()


In [6]:
max_primers = None
max_primers = 1

# EXPERIMENT BLOCK
try:
    log(f'Starting experiment suite: {outputdir}/{foldername}')
    output = []
    for dataset, models in evaluation_sets:
        if dataset is None: continue
        datapath = path.join(datasetdir, dataset)
        primerdir = datapath
        if not path.exists(datapath):
            print(f'Directory not found: {datapath}, skipping')
            continue

        primers = listdir(datapath)[:2]
        # if max_primers: primers = primers[:max_primers]
        # primers = listdir(datapath)

        for (model, checkpoint) in models:
            expname = 'setup'
            start_model(model, checkpoint)

            for name, exp in experiments:
                e = experiment()

                for result in e:
                  result[0] = model
                  result[1] = checkpoint
                  result[2] = dataset

                output += e
            stop_model()
    print(output)
    df = pd.DataFrame(output, columns=cols_gen)
    print(df.describe())
    save_df(df,path.join(outputdir, 'df_gen'))
    

except KeyboardInterrupt:
  print("Terminating...")
  client.pause()
finally:
  stop_model()

print("Done!")

[batch:] Starting experiment suite: ./output/2021-05-19-1
[batch:setup] Starting model: polyphony_rnn (polyphony_rnn)
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /set debug_output False
[batch:setup] /set batch_mode True
[batch:setup] /set trigger_generate 1
[batch:setup] /set batch_unit measures
[batch:setup] /set debug_output False
[batch:setup] /reset 
[batch:setup] /load_bars ./dataset/bach-chorales/260.mid 1
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /save 2021-05-19-1/generation-286077.172276656-1-1-260.mid-0
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /reset 
[batch:setup] /load_bars ./dataset/bach-chorales/260.mid 1
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /save 2021-05-19-1/generation-286077.178463641-1-2-260.mid-0
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /reset 
[batch:setup] /load_bars ./dataset/bach-chorales/260.mid 2
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /save 2021-05-19-1/generation

In [24]:
# Analysis Stage

import os
import glob
df_gen = pd.read_pickle(path.join(outputdir, 'df_gen'))

# For each of the generated models
for model in list(df_gen['model'].unique()):
    # Filter data relative to model
    model_output_data = df_gen.loc[df_gen['model'] == model]

    # Check that output directory exists
    analysisdir = os.path.join(os.path.curdir, 'output', 'samples')
    if not os.path.exists(analysisdir):
        os.mkdir(analysisdir)

    # Prepare baseline (dataset) directory
    dataset = 'bach-chorales'
    datasetoutdir = os.path.join(os.path.abspath(os.path.curdir), 'output', 'dataset')
    if os.path.islink(datasetoutdir):
        os.unlink(datasetoutdir)
        
    os.symlink(os.path.abspath(os.path.join(os.path.curdir, 'dataset', dataset)), datasetoutdir)

    # Iterate Input + Output Lengths
    in_lens = list(model_output_data['in_len'].unique())
    out_lens = list(model_output_data['out_len'].unique())
    # print(model_output_data)
    # print(in_lens)
    # print(out_lens)

    for i in in_lens:
        for j in out_lens:
            # Clean output directory
            for link in glob.glob('output/samples/*'):
                os.unlink(link)

            # Filter output
            df_tmp = df_gen.loc[df_gen['in_len'] == i].loc[df_gen['out_len'] == j]
            outfiles = list(df_tmp['out_file'])

            # Copy output to data directory
            for o, outfile in enumerate(outfiles):

                # Create link to file
                filename = os.path.join("output", f"{outfile}.mid")
                linkname = os.path.join(analysisdir, f'sample-{o}.mid')
                os.symlink(filename, linkname)

            # Process folders
            

            # print(f'model: {model} | checkpoint: {checkpoint} | in_len: {i} | out_len: {j}')
            # print(outfiles)


# path.join(outputdir, df_metrics['out_file'][0] + '.mid')
# ft = get_features_from_file(path.join(outputdir, df_metrics['out_file'][0] + '.mid'))
# print(ft)

df_metrics = df_gen


df_metrics.to_pickle(path.join(outputdir, 'df_metrics'))
print(df_metrics)

           model     checkpoint        dataset   primer  \
0  polyphony_rnn  polyphony_rnn  bach-chorales  179.mid   
1  polyphony_rnn  polyphony_rnn  bach-chorales  179.mid   
2  polyphony_rnn  polyphony_rnn  bach-chorales  179.mid   
3  polyphony_rnn  polyphony_rnn  bach-chorales  179.mid   

                                            out_file          time  in_len  \
0  2021-05-19-1/generation-286077.197133506-1-1-1...  1.142966e-06       1   
1  2021-05-19-1/generation-286077.203463282-1-2-1...  9.929645e-07       1   
2  2021-05-19-1/generation-286077.209735763-2-1-1...  9.009964e-07       2   
3  2021-05-19-1/generation-286077.21703082-2-2-17...  9.660143e-07       2   

   out_len  
0        1  
1        2  
2        1  
3        2  
model: polyphony_rnn | checkpoint: performance_with_dynamics | in_len: 1 | out_len: 1
['2021-05-19-1/generation-286077.197133506-1-1-179.mid-0', '2021-05-19-1/generation-286081.626470812-1-1-179.mid-0', '2021-05-19-1/generation-286082.618925745-1-1

In [8]:
# Plotting phase

df_metrics = pd.read_pickle(path.join(outputdir, 'df_metrics'))

df_metrics.groupby(['model', 'in_len', 'out_len']).describe()

print(output)

IndexError: list index out of range